In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import os


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
DATA_PATH = "/content/drive/MyDrive/Radar_DataSet/data_SAAB_SIRS_77GHz_FMCW.npy"

data = np.load(DATA_PATH, allow_pickle=True)

print("Data type:", type(data))
print("Data shape:", data.shape)   

df = pd.DataFrame(data)
df

Data type: <class 'numpy.ndarray'>
Data shape: (130, 6)


,0,1,2,3,4,5
0,[D1],"[[(-0.0803837295206183-0.024996274181930858j),...","[[62.26415142983489], [62.26415142983489], [62...","[[0.04242135490599999], [0.14313034690999998],...","[[2], [3], [1], [3], [1], [1], [1], [1], [3], ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."
1,[D1],"[[(0.1592469814114204-0.3764412575384074j), (0...","[[116.03773675560139], [116.03773675560139], [...","[[0.04531640558899999], [0.148743200275], [0.2...","[[3], [1], [3], [1], [1], [1], [2], [2], [1], ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."
2,[D1],"[[(-0.36744600235662883-0.17434743507810269j),...","[[75.47169870283017], [75.47169870283017], [75...","[[0.04389842158099999], [0.14448924825099999],...","[[1], [1], [1], [1], [2], [1], [1], [3], [2], ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."
3,[D1],"[[(-0.7918969406816826-0.0388248167751634j), (...","[[145.28302000294806], [145.28302000294806], [...","[[0.04372117357999999], [0.14372117358], [0.24...","[[1], [2], [3], [1], [3], [1], [3], [1], [1], ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."
4,[D1],"[[(0.08721619353018038-0.05631563733996364j), ...","[[22.641509610849056], [22.641509610849056], [...","[[0.03592226153599999], [0.13604042687], [0.23...","[[1], [1], [3], [1], [3], [1], [1], [1], [1], ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."
...,...,...,...,...,...,...
125,[CR],"[[(0.03368695343051483+0.37475747473197485j), ...","[[57.54717026090801], [57.54717026090801], [57...","[[0.145729984258], [0.24614356292700001], [0.3...","[[3], [1], [2], [3], [2], [1], [1], [3], [1], ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."
126,[CR],"[[(-0.28511922708328735+0.32681276930952j), (-...","[[162.26415221108488], [162.26415221108488], [...","[[40.645375488256], [40.744902826920004], [40....","[[1], [1], [3], [3], [3], [1], [1], [2], [2], ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."
127,[CR],"[[(0.2793143954436475-0.19863758289687442j), (...","[[178.30188818543627], [178.30188818543627], [...","[[39.94082612289701], [40.040944288231], [40.1...","[[1], [1], [2], [3], [3], [2], [3], [3], [2], ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."
128,[CR],"[[(0.5004529387476391-0.4401659247452602j), (-...","[[192.45283169221693], [192.45283169221693], [...","[[39.952938069632005], [40.052938069632], [40....","[[2], [3], [1], [3], [1], [2], [3], [2], [2], ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."


In [ ]:
def get_label(label_str):
    drone_classes = ['D1','D2','D3','D4','D5','D6']
    bird_classes = [
        'seagull', 'pigeon', 'raven',
        'black-headed gull',
        'seagull and black-headed gull',
        'heron'
    ]

    if label_str in drone_classes:
        return 1
    elif label_str in bird_classes:
        return 0
    else:
        return None 


In [7]:
class RadarDataset(Dataset):
    def __init__(self, data, split=1):
        self.samples = []

        for row in data:
            label_str = row[0]
            radar_segments = row[1]
            split_info = row[4]

            label = get_label(label_str)
            if label is None:
                continue

            for i in range(radar_segments.shape[1]):
                if split_info[i] == split:
                    segment = radar_segments[:, i]
                    self.samples.append((segment, label))

        print(f"Loaded {len(self.samples)} samples for split {split}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        x, y = self.samples[idx]

        x = np.abs(x)

        x = (x - x.min()) / (x.max() - x.min() + 1e-8)

        x = x.reshape(1, 5, 256)

        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.long)


In [8]:
BATCH_SIZE = 32

train_ds = RadarDataset(data, split=1)
val_ds   = RadarDataset(data, split=2)
test_ds  = RadarDataset(data, split=3)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE)
test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE)


Loaded 52560 samples for split 1
Loaded 7000 samples for split 2
Loaded 7000 samples for split 3


In [9]:
class RadarCNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 1 * 64, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 2)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = RadarCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [11]:
EPOCHS = 20

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{EPOCHS}] - Loss: {avg_loss:.4f}")


Epoch [1/20] - Loss: 0.2066
Epoch [2/20] - Loss: 0.1377
Epoch [3/20] - Loss: 0.1191
Epoch [4/20] - Loss: 0.1102
Epoch [5/20] - Loss: 0.1023
Epoch [6/20] - Loss: 0.0965
Epoch [7/20] - Loss: 0.0880
Epoch [8/20] - Loss: 0.0836
Epoch [9/20] - Loss: 0.0795
Epoch [10/20] - Loss: 0.0740
Epoch [11/20] - Loss: 0.0709
Epoch [12/20] - Loss: 0.0666
Epoch [13/20] - Loss: 0.0628
Epoch [14/20] - Loss: 0.0595
Epoch [15/20] - Loss: 0.0571
Epoch [16/20] - Loss: 0.0540
Epoch [17/20] - Loss: 0.0520
Epoch [18/20] - Loss: 0.0476
Epoch [19/20] - Loss: 0.0464
Epoch [20/20] - Loss: 0.0444


In [12]:
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for x, y in test_loader:
        x = x.to(device)
        outputs = model(x)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()

        y_true.extend(y.numpy())
        y_pred.extend(preds)

print("\nTest Accuracy:", accuracy_score(y_true, y_pred))

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=["Bird", "Drone"]))

print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))



Test Accuracy: 0.9578571428571429

Classification Report:
              precision    recall  f1-score   support

        Bird       0.92      0.78      0.84      1000
       Drone       0.96      0.99      0.98      6000

    accuracy                           0.96      7000
   macro avg       0.94      0.88      0.91      7000
weighted avg       0.96      0.96      0.96      7000


Confusion Matrix:
[[ 776  224]
 [  71 5929]]


In [14]:
SAVE_DIR = "saved_model"
os.makedirs(SAVE_DIR, exist_ok=True)

MODEL_PATH = os.path.join(SAVE_DIR, "radar_drone_bird_cnn.pth")

torch.save({
    "model_state_dict": model.state_dict(),
    "input_shape": (1, 5, 256),
    "classes": {0: "Bird", 1: "Drone"}
}, MODEL_PATH)

print("Model saved at:", MODEL_PATH)


Model saved at: saved_model/radar_drone_bird_cnn.pth


In [ ]:
from google.colab import drive
drive.mount('/content/drive')